In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
load_dotenv();

In [109]:
import base64
import json
import logging
import os
import re
import sys
import time
from functools import partial
from itertools import chain
from pathlib import Path
from uuid import uuid4

import bhutils
import blpapi
import ipyvuetify as v
import ipywidgets as w
import numpy as np
import pandas as pd
import clarion
import QuantLib as ql
import dataframe_image as dfi
import win32com.client as win32
from bqplot import ColorScale, LinearScale, OrdinalColorScale, OrdinalScale
from ipydatagrid import BarRenderer, DataGrid, Expr, TextRenderer, VegaExpr
from IPython.display import display, clear_output
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from blp.blp import BlpParser


from gioutils.clarion_ext import Runner
from gioutils.gui.pandas import millifyp as millify
from gioutils import ezutils as ez
from gioutils.blpw import BlpParser, BlpQuery
from gioutils.clarion_ext import super_clarion
from gioutils.tools import parse_swap_periods
from gioutils.ezutils import (
    bh,
    bhs,
    get_id,
    get_price_info,
)

from gioutils.utils import (
    get_bday,
    get_next_n,
    parse_offset,
    today,
    get_generic_future, 
    get_bbg_fut_chain_ticker,
    get_root
)

pd.options.plotting.backend = "plotly"
pd.options.display.float_format = partial(millify, pct=None, decimal=4)

In [4]:
clarion.server_login()
bq = BlpQuery(timeout=50000, parser=BlpParser(raise_security_errors=False)).start()
# ba = BHSysApi(password="A&3jg5Aj")

2023-02-15 11:57:37,762 - Authenticating...
2023-02-15 11:57:38,353 - Successfully authenticated


In [5]:
super_clarion(clarion)
runner = Runner(clarion)

In [6]:
### check metrics
pd.concat(clarion.metrics()).to_clipboard()

In [63]:
pv = clarion.metric('pv', 'PresentValue', 'Any')
local_pv = clarion.metric('local_pv', 'LocalPresentValue', 'Any')
strike = clarion.metric('strike', 'Strike', 'Any')
notional = clarion.metric('notional', 'Notional', 'Any')
expiry = clarion.metric('expiry', 'ExpiryDate', 'Any')
maturity = clarion.metric('maturity_date', 'MaturityDate', 'Rates', {'Type': 'Bumped'})
start = clarion.metric('start_date', 'StartDate', 'Rates', {'Type': 'Bumped'})
fwd_rate = clarion.metric('fwd_rate', 'Forward', 'Rates')
delta = clarion.metric('delta', 'Delta', 'Rates', {'VolSpotDynamic': 'StickyDelta', 'RiskSpace': 'Par', 'RiskBuckets': 'Default'})
theta = clarion.metric('theta', 'OptionTheta', 'Rates', {'Period': '1'})
price_cents = clarion.metric('price_cents', 'Price', 'Rates', {'Unit': 'Cents'})
price_bps = clarion.metric('price_bps', 'Price', 'Rates', {'Unit': 'BasisPoints'})
price_pct = clarion.metric('price_pct', 'Price', 'Equity', {'Unit': 'PercentLocal'})
price_unit = clarion.metric('price_unit', 'Price', 'Equity', {'Unit': 'Local'})
spot = clarion.metric('spot', 'Spot', 'Rates')
vega = clarion.metric('vega', 'Vega', 'Rates', {'Period': 'Daily'})
atm_vol_day = clarion.metric('atm_vol_day', 'Volatility', 'Rates', {'Type': 'ATM', 'Period': 'Daily'})
atm_vol_ann = clarion.metric('atm_vol_ann', 'Volatility', 'Rates', {'Type': 'ATM', 'Period': 'Annual'})
strike_vol_day = clarion.metric('strike_vol_day', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Daily'})
strike_vol_ann = clarion.metric('strike_vol_ann', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Annual'})
carry_day = clarion.metric('carry_day', 'Carry', 'Credit', {'Horizon': '1d'})

metrics = [
    pv,
    local_pv,
    strike,
    notional,
    expiry,
    maturity,
    start,
    fwd_rate,
    spot,
    delta,
    theta,
    price_cents,
    price_bps,
    price_pct,
    price_unit,
    # vega,
    # atm_vol_day,
    atm_vol_ann,
    # strike_vol_day,
    strike_vol_ann,
    # carry_day
]

### Class

In [223]:
p.get_data(trades, targets)
s2 = p.watch.style.apply(highlight_cond, props='background-color:red; color: white; font-weight: bold;', col='breach', axis=None).format(na_rep='').format(
            partial(millify, zero_rep='-'), 
            subset=p.watch.columns[p.watch.dtypes == 'float64']).set_caption('Trades and triggers')
s1 = p.res.style.format(na_rep='').format(
    partial(millify, zero_rep='-'), 
    subset=p.res.columns[p.res.dtypes == 'float64']
)
indices = p.res.groupby('trade', sort=False).apply(lambda df: df.index[-1])
s1 = s1.set_table_styles({i: [{'selector': '', 'props': 'border-bottom: 3px solid red;'}] for i in indices}, axis=1).set_caption('Details')

2023-02-16 17:20:29,753 - Requesting data
2023-02-16 17:20:41,275 - Request complete
2023-02-16 17:20:41,862 - Response Count Summary - DFRAME:18, IS_ERROR:61


In [224]:
s1

In [225]:
self = p 

In [236]:
clarion_trades = trades

In [237]:
start_date = today()
all_trades = []
i = 0
for key, val in clarion_trades.items():
    if isinstance(val, dict):
        for key2, trade in val.items():
            trade.id = f"{key}{sep}{key2}"
            all_trades.append(trade)
            i += 1
    elif isinstance(val, list):
        for trade in val:
            trade.id = f"{key}{sep}{i}"
            all_trades.append(trade)
            i += 1
self.trades = all_trades
if today() + BDay() - BDay() != today():
    start_date = start_date or today() + BDay() - BDay()
    df = runner.run(metrics, all_trades, start_date=start_date)   
else:
    df = runner.run(metrics, all_trades)

# df = df.set_index('id').loc[[t.id  for t in all trades]]
res = pd.concat([pd.DataFrame(df['id'].str.split(sep).to_list(), columns=['trade', 'id']), runner.df.drop('id', axis=1)], axis=1)
col_order = [*res.columns[~res.columns.isin([metric.label for metric in metrics])], *[metric.label for metric in metrics]]
res['notional'] = res['notional'].astype(float)
res = res.loc[:, col_order]
res = res.set_index('trade').loc[[*trades], :].reset_index().replace('', np.nan).infer_objects()
self.data = res

2023-02-16 17:26:17,332 - Requesting data
2023-02-16 17:26:28,577 - Request complete
2023-02-16 17:26:29,113 - Response Count Summary - DFRAME:18, IS_ERROR:61


In [246]:
ress = []
for index, group in self.data.groupby('trade', sort=False):
    # display(group)

    target_mult = targets.get(index, {}).get('mult', {})
    totals = pd.Series(
        {
            'trade': index,
            'id': 'total',
            'price_bps': (group['price_bps'] * group['notional']).sum() / group['notional'].abs().min(),
            'price_unit': (group['price_unit'] * group['notional']).sum() / group['notional'].abs().min(),
            'size': ' x '.join([millify(x) for x in group['notional']]),
            'pv': group['pv'].sum(),
            'local_pv': group['local_pv'].sum(),
            'delta': group['delta'].sum(),
            'fwd_rate': (group['fwd_rate'] * np.sign(group['notional'])).sum() if not target_mult.get("fwd_rate") else (group['fwd_rate'] * np.sign(group['delta']) * np.asarray(target_mult.get("fwd_rate"))).sum(),
        }
    ).to_frame().T
    r = pd.concat([totals, group])[['size', *col_order]]
    ress.append(r)

{'var': 'fwd_rate', 'op': '>=', 'target': 5.5, 'notes': '', 'mult': {'fwd_rate': (1, 2, 1)}}
{'var': 'fwd_rate', 'op': '>=', 'target': 5.5, 'notes': '', 'mult': {'fwd_rate': (1, 2, 1)}}
{'var': 'fwd_rate', 'op': '>=', 'target': 5.5, 'notes': '', 'mult': {'fwd_rate': (1, 2, 1)}}
{'var': 'fwd_rate', 'op': '>=', 'target': 5.5, 'notes': '', 'mult': {'fwd_rate': (1, 2, 1)}}
{'var': 'fwd_rate', 'op': '>=', 'target': 5.5, 'notes': '', 'mult': {'fwd_rate': (1, 2, 1)}}
{'var': 'fwd_rate', 'op': '>=', 'target': 5.5, 'notes': '', 'mult': {'fwd_rate': (1, 2, 1)}}


In [249]:
group['fwd_rate']

13   2.8895
14   2.7692
15   2.8524
Name: fwd_rate, dtype: float64

In [250]:
(group['fwd_rate'] * np.sign(group['delta']) * np.asarray(target_mult.get("fwd_rate"))).sum()

-0.20355719588232368

In [247]:
index

'eur6m_fly_2y2y_4y2y_6y2y'

In [253]:
sep = '__'


class Pricer:
    
    def get_data(self, clarion_trades, targets, start_date=None):
        start_date = start_date or today() + BDay() - BDay()
        all_trades = []
        i = 0
        for key, val in clarion_trades.items():
            if isinstance(val, dict):
                for key2, trade in val.items():
                    trade.id = f"{key}{sep}{key2}"
                    all_trades.append(trade)
                    i += 1
            elif isinstance(val, list):
                for trade in val:
                    trade.id = f"{key}{sep}{i}"
                    all_trades.append(trade)
                    i += 1
        self.trades = all_trades
        if today() + BDay() - BDay() != today():
            start_date = start_date or today() + BDay() - BDay()
            df = runner.run(metrics, all_trades, start_date=start_date)   
        else:
            df = runner.run(metrics, all_trades)
        
        # df = df.set_index('id').loc[[t.id  for t in all trades]]
        res = pd.concat([pd.DataFrame(df['id'].str.split(sep).to_list(), columns=['trade', 'id']), runner.df.drop('id', axis=1)], axis=1)
        col_order = [*res.columns[~res.columns.isin([metric.label for metric in metrics])], *[metric.label for metric in metrics]]
        res['notional'] = res['notional'].astype(float)
        res = res.loc[:, col_order]
        res = res.set_index('trade').loc[[*trades], :].reset_index().replace('', np.nan).infer_objects()
        self.data = res
        
        ress = []
        for index, group in res.groupby('trade', sort=False):
            # display(group)
            
            target_mult = targets.get(index, {}).get('mult', {})
            totals = pd.Series(
                {
                    'trade': index,
                    'id': 'total',
                    'price_bps': (group['price_bps'] * group['notional']).sum() / group['notional'].abs().min(),
                    'price_unit': (group['price_unit'] * group['notional']).sum() / group['notional'].abs().min(),
                    'size': ' x '.join([millify(x) for x in group['notional']]),
                    'pv': group['pv'].sum(),
                    'local_pv': group['local_pv'].sum(),
                    'delta': group['delta'].sum(),
                    'fwd_rate': (group['fwd_rate'] * np.sign(group['notional'])).sum() if not target_mult.get("fwd_rate") else (group['fwd_rate'] * np.sign(group['delta']) * np.asarray(target_mult.get("fwd_rate"))).sum(),
                }
            ).to_frame().T
            r = pd.concat([totals, group])[['size', *col_order]]
            ress.append(r)
        ress = pd.concat(ress)
        
        self.res = ress.infer_objects().reset_index(drop=True).drop('period', axis=1)
        total = ress.query('id=="total"').dropna(axis=1).drop("id", axis=1).set_index('trade')
        # ress._total = total
        
        targets_ = pd.DataFrame(targets).T
        if 'mult' in targets_:
            targets_ = targets_.drop('mult', axis=1)
        exprs = {}
        for col in total:
            exprs[col] = bh.bhTsCalcExpression(
                resultnames=range(len(targets_)),  # if omitted it doesn't work with multiple expressions at once, 
                expressions=[*targets_.index], 
                varnamesorhndl=[*total.index], 
                varvalues=total[col]
            )
        exprs = pd.DataFrame(exprs, index = targets_.index)
        
        
        watch = pd.concat([exprs, targets_], axis=1) # total.merge(exprs, right_index=True, left_index=True, how='right')
        # display(exprs)
        # display(total)
        # display(watch)
        
        watch = watch.assign(breach=watch.apply(lambda row: eval(f"{row[row['var']]} {row['op']} {row['target']}") if not pd.isnull(row['target']) else False, axis=1))
        self.watch = watch.infer_objects()

In [91]:
def highlight_cond(df, props, col):
    d = pd.DataFrame(None, df.index, df.columns)
    d.loc[df[col]] = props
    return d 

### TRADES

In [202]:
trades = dict(
    eur_bear_flat_9m1y_9m2y = [
        clarion.instruments.RatesSwaption(
            currency='EUR',
            instrument_name='EURIBOR A 3M',
            expiry='9m',
            tenor='1y',
            strike='3.65',
            swaption_type='Pay',
            notional=2e9,
            settle_type='Cash Price',
            collateral_currency='USD',
            pay_settle='Spot',
            #id='eur_9m_bear_flat'
        ),
        clarion.instruments.RatesSwaption(
            currency='EUR',
            instrument_name='EURIBOR A 6M',
            expiry='9m',
            tenor='2y',
            strike='3.4',
            swaption_type='Pay',
            notional=-1e9,
            settle_type='Cash Price',
            collateral_currency='USD',
            pay_settle='Spot',
            #id='eur_9m_bear_flat'
        ),

        clarion.instruments.RatesSwaption(
            currency='EUR',
            instrument_name='EURIBOR A 6M',
            expiry='9m',
            tenor='2y',
            strike='2.8',
            swaption_type='Rec',
            notional=1e9,
            settle_type='Cash Price',
            collateral_currency='USD',
            pay_settle='Spot',
            #id='eur_9m_bear_flat'
        ),

        clarion.instruments.RatesSwaption(
            currency='EUR',
            instrument_name='EURIBOR A 6M',
            expiry='9m',
            tenor='2y',
            strike='2.0',
            swaption_type='Rec',
            notional=-1e9,
            settle_type='Cash Price',
            collateral_currency='USD',
            pay_settle='Spot',
            #id='eur_9m_bear_flat'
        ),
    ],
    eur3m_bull_steep_15m1y_15m1y1y = [
            clarion.instruments.RatesSwaption(
                currency='EUR',
                instrument_name='EURIBOR A 3M',
                expiry='2024-05-03',
                tenor='1y',
                strike='2.425',
                swaption_type='Rec',
                notional=-1e9,
                settle_type='Cash Price',
                collateral_currency='USD',
                pay_settle='Spot',
                #id='eur_9m_bear_flat'
            ),
            clarion.instruments.RatesSwaption(
                currency='EUR',
                instrument_name='EURIBOR A 3M',
                expiry='2024-05-03',
                tenor='1yx1y',
                strike='2.0',
                swaption_type='Rec',
                notional=1e9,
                settle_type='Cash Price',
                collateral_currency='USD',
                pay_settle='Spot',
                #id='eur_9m_bear_flat'
            ),
    ],
    eur6m_bull_steep_30m2y_30m7y = [
            clarion.instruments.RatesSwaption(
                currency='EUR',
                instrument_name='EURIBOR A 6M',
                expiry='30m',
                tenor='2y',
                strike='1.8',
                swaption_type='Rec',
                notional=-400e6,
                settle_type='Cash Price',
                collateral_currency='USD',
                pay_settle='Spot',
                #id='eur_9m_bear_flat'
            ),
            clarion.instruments.RatesSwaption(
                currency='EUR',
                instrument_name='EURIBOR A 6M',
                expiry='30m',
                tenor='7y',
                strike='1.8',
                swaption_type='Rec',
                notional=120e6,
                settle_type='Cash Price',
                collateral_currency='USD',
                pay_settle='Spot',
                #id='eur_9m_bear_flat'
            ),
    ],
    sofr_bull_steep_2y7y_2y30y = [
            clarion.instruments.RatesSwaption(
                currency='USD',
                instrument_name='SOFR A',
                expiry='2y',
                tenor='7y',
                strike='2.3',
                swaption_type='Rec',
                notional=-160e6,
                settle_type='Cash Price',
                collateral_currency='USD',
                pay_settle='Spot',
                #id='eur_9m_bear_flat'
            ),
            clarion.instruments.RatesSwaption(
                currency='USD',
                instrument_name='SOFR A',
                expiry='2y',
                tenor='30y',
                strike='2.4',
                swaption_type='Rec',
                notional=36e6,
                settle_type='Cash Price',
                collateral_currency='USD',
                pay_settle='Spot',
                #id='eur_9m_bear_flat'
            ),
    ],
    sonia_rec_fly_18m1y = [
            clarion.instruments.RatesSwaption(
                currency='GBP',
                instrument_name='SONIA A',
                expiry='18m',
                tenor='1y',
                strike='2.5',
                swaption_type='Rec',
                notional=800e6,
                settle_type='Cash Price',
                collateral_currency='USD',
                pay_settle='Spot',
                #id='eur_9m_bear_flat'
            ),
            clarion.instruments.RatesSwaption(
                currency='GBP',
                instrument_name='SONIA A',
                expiry='18m',
                tenor='1y',
                strike='1.5',
                swaption_type='Rec',
                notional=-1600e6,
                settle_type='Cash Price',
                collateral_currency='USD',
                pay_settle='Spot',
                #id='eur_9m_bear_flat'
            ),
            clarion.instruments.RatesSwaption(
                currency='GBP',
                instrument_name='SONIA A',
                expiry='18m',
                tenor='1y',
                strike='1.0',
                swaption_type='Rec',
                notional=800e6,
                settle_type='Cash Price',
                collateral_currency='USD',
                pay_settle='Spot',
                #id='eur_9m_bear_flat'
            ),
    ],
    eur6m_fly_2y2y_4y2y_6y2y = [
        clarion.instruments.RatesSwap(
            currency='EUR',
            instrument_name='EURIBOR A 6M',
            strike='a',
            start_date='2y',
            tenor='2y',
            swap_type='Rec',
            clearing_house='LCH',
        ),
        clarion.instruments.RatesSwap(
            currency='EUR',
            instrument_name='EURIBOR A 6M',
            strike='a',
            start_date='4y',
            swap_type='Pay',
            tenor='2y',
            clearing_house='LCH',
        ),
        clarion.instruments.RatesSwap(
            currency='EUR',
            instrument_name='EURIBOR A 6M',
            strike='a',
            start_date='6y',
            tenor='2y',
            swap_type='Rec',
            clearing_house='LCH',
        )
    ],
    
    spx_nov23_put_spread = [
        clarion.instruments.EquityCashOption(
            instrument_name='SPX',
            contract_type='US',
            expiry='2023-11-17',
            strike=3000,
            option_type='Put',
            exercise_type='European',
            quantity=600

        ),
        clarion.instruments.EquityCashOption(
            instrument_name='SPX',
            contract_type='US',
            expiry='2023-11-17',
            strike=2900,
            option_type='Put',
            exercise_type='European',
            quantity=-600

        )
    ]
)

### TARGETS

In [222]:
targets = {
    'eur_bear_flat_9m1y_9m2y': {
        "var": "local_pv",
        "op":  "<=", 
        "target": 1.0e6, 
        "notes": "we BUY this structure in 500m on the first leg for 275k or better"
    },
    'eur3m_bull_steep_15m1y_15m1y1y': {
        "var": "local_pv", 
        "op":  ">=",
        "target": 180e3,
        "notes": "we SELL 1y1y and receive 180k"
    },
    'eur6m_bull_steep_30m2y_30m7y': {
        "var": "local_pv", 
        "op":  ">=",
        "target": -65e3,
        "notes": "we SELL 7y pay 65k in 400x120"
    },
    'sofr_bull_steep_2y7y_2y30y': {
        "var": "local_pv", 
        "op":  ">=",
        "target": -680e3,
        "notes": "we SELL 30y pay 680k"
    },
    'sonia_rec_fly_18m1y': {
        "var": "price_bps", 
        "op":  "<=",
        "target": 8,
        "notes": "we SELL 1.5 strike (middle) for 8.5 or better"
    },
    'spx_nov23_put_spread': {
        "var": "price_unit", 
        "op":  ">=",
        "target": 5.5,
        "notes": ""
    },
    # 'sofr_bull_steep_2y7y_2y30y - sofr_bull_steep_2y7y_2y30y': {
    #     # "var": "local_pv", 
    #     # "op":  ">=",
    #     # "target": 2e6,
    #     # "notes": "we sell 30y pay -640k"
    # },
    'eur6m_fly_2y2y_4y2y_6y2y': {
        "var": "fwd_rate", 
        "op":  ">=",
        "target": 5.5,
        "notes": "",
        'mult': {"fwd_rate": (1, 2, 1)}
    },
}
targets_ = pd.DataFrame(targets).T

In [254]:
p = Pricer()

In [255]:
out = w.Output()

In [189]:
last_update = time.ctime()
last_email_sent = time.time()
while True:
    try:
        p.get_data(trades, targets)
        last_update = time.ctime()
        s2 = p.watch.style.apply(highlight_cond, props='background-color:red; color: white; font-weight: bold;', col='breach', axis=None).format(na_rep='').format(
            partial(millify, zero_rep='-'), 
            subset=p.watch.columns[p.watch.dtypes == 'float64']).set_caption('Trades and triggers')
        s1 = p.res.style.format(na_rep='').format(
            partial(millify, zero_rep='-'), 
            subset=p.res.columns[p.res.dtypes == 'float64']
        )
        indices = p.res.groupby('trade', sort=False).apply(lambda df: df.index[-1])
        s1 = s1.set_table_styles({i: [{'selector': '', 'props': 'border-bottom: 3px solid red;'}] for i in indices}, axis=1).set_caption('Details')
        clear_output()
        with out:
            out.clear_output()
            display(last_update)
            display(s2)
            display(s1)
        display(out)
        
        if ((time.time() - last_email_sent) / 3600) >= 1:
            outlook = win32.Dispatch('outlook.application')
            mail = outlook.CreateItem(0)
            mail.To = 'giovambattista.perciaccante@brevanhoward.com; gioxc@hotmail.it' # menashe.banit@brevanhoward.com'
            # mail.To = 'giovambattista.perciaccante@brevanhoward.com'
            mail.Subject = f'QUICK UPDATE: Trade Watch {pd.Timestamp.now(): %d%b%y %H:%M}'
            file_name = 'trades'
            html_body = ''
            for i, s in enumerate([s2, s1]):
                file_name_ = f"{file_name}{i}.png"
                dfi.export(s, file_name_)
                image_file = open(file_name_, 'rb').read()
                encoded_image = base64.b64encode(image_file).decode("utf-8")
                html_body += f'{"<br>" * (5 if i > 0 else 1)}<img src="data:image/png;base64,{encoded_image}"/>'
                mail.Attachments.Add(f"{str(Path().resolve() / file_name_)}")
            mail.HTMLBody = html_body
            # mail.HTMLBody = d[1].reset_index().to_csv(header=False, index=False)
            # To attach a file to the email (optional)
            # attachment  = "Path to the attachment"
            mail.Send()
            last_email_sent = time.time()
        # break
        time.sleep(30)
    except:
        clear_output()
        display(f"BROKEN: {last_update}")
        display(s2)
        display(s1)
        time.sleep(5)

'BROKEN: Thu Feb 16 13:38:02 2023'

KeyboardInterrupt: 

In [ ]:
display(s2)
display(s1)

### Send EMAIL

In [256]:
p.get_data(trades, targets)

2023-02-16 17:51:14,898 - Requesting data
2023-02-16 17:51:26,130 - Request complete
2023-02-16 17:51:26,652 - Response Count Summary - DFRAME:18, IS_ERROR:61


In [259]:
s2 = p.watch.style.apply(highlight_cond, props='background-color:red; color: white; font-weight: bold;', col='breach', axis=None).format(na_rep='').format(
            partial(millify, decimal=3,  pct=None, zero_rep='-'), 
            subset=p.watch.columns[p.watch.dtypes == 'float64']).set_caption('Trades and triggers')
s1 = p.res.style.format(na_rep='').format(
    partial(millify, decimal=3, pct=None, zero_rep='-'), 
    subset=p.res.columns[p.res.dtypes == 'float64']
)
indices = p.res.groupby('trade', sort=False).apply(lambda df: df.index[-1])
s1 = s1.set_table_styles({i: [{'selector': '', 'props': 'border-bottom: 3px solid red;'}] for i in indices}, axis=1).set_caption('Details')

In [260]:
s2

In [251]:
s1

In [213]:
p.res.set_index('trade')

size     id periodType  \
trade                                                                    
eur_bear_flat_9m1y_9m2y          2b x -1b x 1b x -1b  total        NaN   
eur_bear_flat_9m1y_9m2y                          NaN      0      daily   
eur_bear_flat_9m1y_9m2y                          NaN      1      daily   
eur_bear_flat_9m1y_9m2y                          NaN      2      daily   
eur_bear_flat_9m1y_9m2y                          NaN      3      daily   
eur3m_bull_steep_15m1y_15m1y1y              -1b x 1b  total        NaN   
eur3m_bull_steep_15m1y_15m1y1y                   NaN      4      daily   
eur3m_bull_steep_15m1y_15m1y1y                   NaN      5      daily   
eur6m_bull_steep_30m2y_30m7y            -400m x 120m  total        NaN   
eur6m_bull_steep_30m2y_30m7y                     NaN      6      daily   
eur6m_bull_steep_30m2y_30m7y                     NaN      7      daily   
sofr_bull_steep_2y7y_2y30y               -160m x 36m  total        NaN   
sofr_bull_steep_2y7y_2y30y                       NaN      8      daily   
sofr_bull_steep_2y7y_2y30y                       NaN      9      daily   
sonia_rec_fly_18m1y             800m x -1.60b x 800m  total        NaN   
sonia_rec_fly_18m1y                              NaN     10      daily   
sonia_rec_fly_18m1y                              NaN     11      daily   
sonia_rec_fly_18m1y                              NaN     12      daily   
eur6m_fly_2y2y_4y2y_6y2y                  1 x -1 x 1  total        NaN   
eur6m_fly_2y2y_4y2y_6y2y                         NaN     13      daily   
eur6m_fly_2y2y_4y2y_6y2y                         NaN     14      daily   
eur6m_fly_2y2y_4y2y_6y2y                         NaN     15      daily   
spx_nov23_put_spread                      600 x -600  total        NaN   
spx_nov23_put_spread                             NaN     16      daily   
spx_nov23_put_spread                             NaN     17      daily   

                                                                      description  \
trade                                                                               
eur_bear_flat_9m1y_9m2y                                                       NaN   
eur_bear_flat_9m1y_9m2y         EUR EURIBOR A 3M 9m 1y 3.65 Payer USD CashPric...   
eur_bear_flat_9m1y_9m2y         EUR EURIBOR A 6M 9m 2y 3.4 Payer USD CashPrice...   
eur_bear_flat_9m1y_9m2y         EUR EURIBOR A 6M 9m 2y 2.8 Receiver USD CashPr...   
eur_bear_flat_9m1y_9m2y         EUR EURIBOR A 6M 9m 2y 2 Receiver USD CashPric...   
eur3m_bull_steep_15m1y_15m1y1y                                                NaN   
eur3m_bull_steep_15m1y_15m1y1y  EUR EURIBOR A 3M 03-May-24 1y 2.425 Receiver U...   
eur3m_bull_steep_15m1y_15m1y1y  EUR EURIBOR A 3M 03-May-24 1yx1y 2 Receiver US...   
eur6m_bull_steep_30m2y_30m7y                                                  NaN   
eur6m_bull_steep_30m2y_30m7y    EUR EURIBOR A 6M 30m 2y 1.8 Receiver USD CashP...   
eur6m_bull_steep_30m2y_30m7y    EUR EURIBOR A 6M 30m 7y 1.8 Receiver USD CashP...   
sofr_bull_steep_2y7y_2y30y                                                    NaN   
sofr_bull_steep_2y7y_2y30y       USD SOFR A 2y 7y 2.3 Receiver USD CashPrice Spot   
sofr_bull_steep_2y7y_2y30y      USD SOFR A 2y 30y 2.4 Receiver USD CashPrice Spot   
sonia_rec_fly_18m1y                                                           NaN   
sonia_rec_fly_18m1y             GBP SONIA A 18m 1y 2.5 Receiver USD CashPrice ...   
sonia_rec_fly_18m1y             GBP SONIA A 18m 1y 1.5 Receiver USD CashPrice ...   
sonia_rec_fly_18m1y              GBP SONIA A 18m 1y 1 Receiver USD CashPrice Spot   
eur6m_fly_2y2y_4y2y_6y2y                                                      NaN   
eur6m_fly_2y2y_4y2y_6y2y                                   EUR EURIBOR A 6M 2y 2y   
eur6m_fly_2y2y_4y2y_6y2y                                   EUR EURIBOR A 6M 4y 2y   
eur6m_fly_2y2y_4y2y_6y2y                                   EUR EURIBOR A 6M 6y 2y   
spx_nov23

In [176]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'giovambattista.perciaccante@brevanhoward.com; menashe.banit@brevanhoward.com'
# mail.To = 'giovambattista.perciaccante@brevanhoward.com'
mail.Subject = f'QUICK UPDATE: Trade Watch {pd.Timestamp.now(): %d%b%y %H:%M}'
file_name = 'trades'
html_body = ''
for i, s in enumerate([s2, s1]):
    file_name_ = f"{file_name}{i}.png"
    dfi.export(s, file_name_)
    image_file = open(file_name_, 'rb').read()
    encoded_image = base64.b64encode(image_file).decode("utf-8")
    html_body += f'{"<br>" * (5 if i > 0 else 1)}<img src="data:image/png;base64,{encoded_image}"/>'
    mail.Attachments.Add(f"{str(Path().resolve() / file_name_)}")
mail.HTMLBody = html_body
# mail.HTMLBody = d[1].reset_index().to_csv(header=False, index=False)
# To attach a file to the email (optional)
# attachment  = "Path to the attachment"
mail.Send()